In [1]:
#导入numpy，可以提供更高效的数值计算
import numpy as np
#导入pandas，支持数据帧的便捷库
import pandas as pd
#导入modelselection模块中的train_test_split函数，包含许多实用程序
from sklearn.model_selection import train_test_split
#导入整个预处理模块，包含用于缩放、转换和整理数据的实用程序
from sklearn import preprocessing
#导入随机森林模型，这段代码中只专注于训练随机森林并调整其参数。
from sklearn.ensemble import RandomForestRegressor
#导入工具以执行交叉验证
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
#导入一些指标，以便稍后用于评估模型性能
from sklearn.metrics import mean_squared_error, r2_score
#导入一种保留模型以供将来使用的方法
import joblib

In [2]:
#加载数据
dataset_url = 'winequality/winequality-red.csv'
data = pd.read_csv(dataset_url)
#查看数据的前五行
print( data.head() )

  fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
0   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                                     
1   7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5                                                                                                                     
2  7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...                                                                                                                     
3  11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...                                                                                                                     
4   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                  

In [3]:
data = pd.read_csv(dataset_url, sep=';')
print(data.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [4]:
print(data.shape)
print(data.describe())

(1599, 12)
       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.0000

In [5]:
#将quality列为标签，赋值给y
y = data.quality
#删除data数据框中的quality列，剩余部分作为特征，赋值给X
#axis=0表示沿着横向维度，也就是对行进行操作
#axis=1表示沿着纵向维度，也就是对列进行操作
X = data.drop('quality', axis = 1)
#使用sklearn中的train_test_split函数将X和y划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

In [6]:
#在训练数据X_train上拟合StandardScaler()
scaler = preprocessing.StandardScaler().fit(X_train)
#使用上一条语句中的参数将X_train进行标准化
X_train_scaled = scaler.transform(X_train)
#打印标准差和方差
print (X_train_scaled.mean(axis=0))
print (X_train_scaled.std(axis=0))

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.77772766e-18 -6.38877362e-17 -4.16659149e-18 -1.20753377e-13
 -8.70817622e-16 -4.08325966e-16 -1.16664562e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
X_test_scaled = scaler.transform(X_test)
print (X_test_scaled.mean(axis=0))
print (X_test_scaled.std(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[1.02160495 1.00135689 0.97456598 0.91099054 0.86716698 0.94193125
 1.03673213 1.03145119 0.95734849 0.83829505 1.0286218 ]


In [8]:
#构建一个机器学习流程（pipeline）
pipeline = make_pipeline(preprocessing.StandardScaler(),
                         RandomForestRegressor(n_estimators=100,
                                               random_state=123))

In [9]:
#列出可调超参数
print( pipeline.get_params() )

{'memory': None, 'steps': [('standardscaler', StandardScaler()), ('randomforestregressor', RandomForestRegressor(random_state=123))], 'verbose': False, 'standardscaler': StandardScaler(), 'randomforestregressor': RandomForestRegressor(random_state=123), 'standardscaler__copy': True, 'standardscaler__with_mean': True, 'standardscaler__with_std': True, 'randomforestregressor__bootstrap': True, 'randomforestregressor__ccp_alpha': 0.0, 'randomforestregressor__criterion': 'squared_error', 'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 1.0, 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__max_samples': None, 'randomforestregressor__min_impurity_decrease': 0.0, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2, 'randomforestregressor__min_weight_fraction_leaf': 0.0, 'randomforestregressor__n_estimators': 100, 'randomforestregressor__n_jobs': None, 'randomforestregressor__oob_score': False, 'rando

In [10]:
#设置超参数
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [11]:
#GridSearch本质上就是对超参数的所有可能的排列执行交叉验证
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
#拟合和调整模型
clf.fit(X_train, y_train)
#输出最佳参数集
print( clf.best_params_ )

E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomF

E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
E:\Program Files\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomF

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'sqrt'}


In [12]:
print( clf.refit )


True


In [13]:
#使用训练好的分类模型clf对测试集X_test进行预测,结果存储在y_pred中。
y_pred = clf.predict(X_test)

In [14]:
#将真实标签y_test和预测结果y_pred输入r2_score函数,计算R squared(R方)值,并打印结果。
print( r2_score(y_test, y_pred) )
# 0.4712595193413647
#将真实标签y_test和预测结果y_pred输入mean_squared_error函数,计算均方误差(MSE),并打印结果。
print( mean_squared_error(y_test, y_pred) )
# 0.34118218749999996

0.4712595193413647
0.34118218749999996


In [15]:
#将训练好的模型clf序列化后保存到文件rf_regressor.pkl中
joblib.dump(clf, 'rf_regressor.pkl')
#从文件rf_regressor.pkl中加载序列化后的模型,并赋值给clf2。
clf2 = joblib.load('rf_regressor.pkl')
 #用现成的模型预测数据
clf2.predict(X_test)

array([6.44, 5.76, 4.99, 5.63, 6.27, 5.61, 4.89, 4.74, 5.03, 5.83, 5.26,
       5.68, 5.85, 5.11, 5.86, 5.6 , 6.52, 5.8 , 5.75, 6.96, 5.42, 5.68,
       5.1 , 6.02, 5.95, 5.03, 5.33, 5.22, 6.04, 5.9 , 5.89, 6.48, 6.01,
       5.03, 4.95, 5.95, 5.05, 6.1 , 5.15, 6.05, 4.92, 5.92, 6.71, 5.08,
       6.23, 5.33, 5.49, 5.53, 5.22, 6.4 , 6.05, 5.25, 5.81, 5.22, 5.6 ,
       5.64, 5.38, 5.38, 5.04, 5.12, 5.28, 5.24, 5.03, 5.8 , 6.01, 5.33,
       6.43, 5.03, 5.14, 6.64, 5.78, 5.84, 5.09, 5.06, 5.34, 6.01, 5.29,
       5.06, 5.21, 5.26, 6.33, 5.66, 6.11, 6.33, 5.09, 5.94, 6.55, 6.3 ,
       5.74, 5.71, 5.86, 5.37, 6.3 , 5.69, 5.64, 5.78, 6.68, 6.77, 5.53,
       6.78, 5.16, 5.34, 5.1 , 6.49, 5.07, 4.69, 5.61, 5.05, 5.66, 5.9 ,
       5.92, 5.47, 6.01, 5.28, 4.92, 5.19, 5.92, 5.13, 5.05, 6.01, 5.87,
       5.06, 5.79, 5.99, 5.26, 5.43, 5.28, 5.89, 5.57, 5.45, 5.71, 6.01,
       5.2 , 5.33, 5.08, 6.38, 5.  , 5.22, 6.71, 5.4 , 5.13, 5.11, 5.62,
       6.04, 5.34, 5.34, 5.07, 6.48, 5.71, 5.12, 5.